<a href="https://colab.research.google.com/github/IrinaBolshagina/Comics/blob/main/3_Generating_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I generate comic book style images using image-to-image [Stable Diffusion](https://huggingface.co/stabilityai/sdxl-turbo) model

1. Create a pipleline for stable diffusion
2. Create a prompt
3. Put prompt and initial image into pipeline to get generated picture (I put pictures one by one, because I couldn't find the way to load batch of pictures into pipeline)

In [1]:
!pip install diffusers transformers accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [10]:
import requests
from PIL import Image
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

In [11]:
img_urls = [
    'https://expertphotography.b-cdn.net/wp-content/uploads/2022/06/Birthday-Photoshoot-Ideas-surprise.jpg',
    'https://img.freepik.com/free-photo/indoor-portrait-amazing-female-model-trendy-t-shirt-touching-her-long-shiny-hair_197531-6512.jpg',
    'https://expertphotography.b-cdn.net/wp-content/uploads/2022/06/Birthday-Photoshoot-Ideas-pool-party.jpg',
    'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQEWGvDGqJ4FvpuUWT0PXxgoY9utJd-GtR9A&usqp=CAU',
    'https://play-lh.googleusercontent.com/on623rjZr4fRqI1YCTfOthkLzS6RSTJY5WhGWj9iO5EpWJDWNnmBuW_dcreZVoyx4Q'

]

In [70]:
def download_images_from_urls(urls):
    images = []
    for url in urls:
        response = requests.get(url)
        img = Image.open(requests.get(url, stream=True).raw).convert('RGB')
        images.append(img.resize((4096, 4096)))
    return images


In [71]:
downloaded_images = download_images_from_urls(img_urls)

##1. Create a pipleline for stable diffusion

In [63]:
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

## 2. Create a prompt

In [62]:
init_image = load_image("https://expertphotography.b-cdn.net/wp-content/uploads/2022/06/Birthday-Photoshoot-Ideas-surprise.jpg")

prompt1 = "Comicbook: comic, graphic illustration, comic art, graphic novel art, vibrant, highly detailed, colored, 2d minimalistic"
prompt2 = "Comicbook: a woman in a white shirt and black pants with red hearts, graphic illustration, graphic novel art, vibrant, pretty, colored, 2d minimalistic"
prompt3 = "a woman in a white shirt and black pants with red hearts, comic art, graphic illustration, graphic novel art, vibrant, pretty, colored, 2d minimalistic"
prompt4 = "comicbook style, graphic illustration, graphic novel art, vibrant, pretty, colored, 2d minimalistic"
prompt5 = "a man in a suit and tie is dancing with his friends, comicbook style, graphic illustration, graphic novel art, vibrant, pretty, colored, 2d minimalistic"
negative_prompt="ugly, deformed, disfigured, poor details, bad anatomy"

In [ ]:
# test the prompt

init_image = load_image("https://img.freepik.com/free-photo/indoor-portrait-amazing-female-model-trendy-t-shirt-touching-her-long-shiny-hair_197531-6512.jpg")
img_description = "a man in a suit and tie is dancing with his friends"

image1 = pipeline(prompt4, image=init_image, strength=0.7, guidance_scale=8.0, negative_prompt=negative_prompt).images[0]
image2 = pipeline(img_description + ',' + prompt4, image=init_image, strength=0.7, guidance_scale=8.0, negative_prompt=negative_prompt).images[0]

make_image_grid([init_image, image1, image2], rows=1, cols=3)

## 3. Pass prompt and image to pipeline to get generated image

In [46]:
def generate_images(img_list, prompt):
  gen_img_list = []
  for init_image in img_list:
    image = pipeline(prompt, image=init_image, strength=0.7, guidance_scale=8.0, negative_prompt=negative_prompt).images[0]
    gen_img_list.append(image)
  return gen_img_list

In [68]:
import requests
import torch
from PIL import Image
from diffusers import AutoPipelineForImage2Image
from torchvision import transforms

def generate_images(img_list, prompt):
    # Define the transformation pipeline
    transform = transforms.Compose([
        transforms.Resize((4096, 4096)),
        transforms.ToTensor(),
    ])

    # Convert the list of images to tensors
    tensor_images = [transform(image) for image in img_list]

    # Stack the tensors to create a batch
    tensor_batch = torch.stack(tensor_images)

    # Pass the entire tensor batch to the model at once
    generated_images = pipeline(prompt, image=tensor_batch).images

    # Convert the generated images tensor back to a list
    gen_img_list = [image for image in generated_images]

    return gen_img_list

In [69]:
generated_images = generate_images(downloaded_images, prompt4)
for img in generated_images:
  display(img)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 GiB. GPU 0 has a total capacty of 14.75 GiB of which 9.81 GiB is free. Process 6388 has 4.93 GiB memory in use. Of the allocated memory 4.22 GiB is allocated by PyTorch, and 587.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF